<a href="https://colab.research.google.com/github/yoshihiroo/programming-workshop/blob/master/QC4U_2022/qc4uchapter3_cirq_English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QC4U Day3 Cirq porting
2022.10.15 updated.

This is an attempt to porting of the [QC4U](https://altema.is.tohoku.ac.jp/QC4U/) code written by Prof. Ohzeki of Tohoku University into Cirq for my recap and understanding. I am stealing with pride the almost all text of the explanation from the original site. (The article has been published with Prof. Ohzeki's permission.)

[The original code of Day 3](https://colab.research.google.com/gist/mohzeki222/03914646f0c7fb8bc4826cddbd44ac23/qc4uchapter3.ipynb)

# Installing Cirq

Let's continue to use Cirq provided by Google as before.

In [ ]:
pip install cirq

In this time, we will introduce machine learning using quantum computers.

In [ ]:
import cirq

### Recap

So far we have learned H, X, Z, and the control Z-gate.
Each of them is characterized by H = superposition, X = inversion, and Z = scratch (negative sign for only |1>).
And the control Z gate was to scratch only|11>.
Once again, I think that Z is great for scratching probability amplitudes.
Let's look again at Z acting on two qubits.
To see what kind of changes we can make at once here, let's apply Hadamard first.

Translated with www.DeepL.com/Translator (free version)

In [ ]:
qc = cirq.Circuit()
q = cirq.LineQubit.range(2)

qc.append(cirq.H.on_each(q))
qc.append(cirq.Z(q[0]))

sim = cirq.Simulator()
res = sim.simulate(qc)

If you want to see what the circuit looks like, run print(qc).

In [ ]:
print(qc)

In [ ]:
print(res.final_state_vector.round(5))

As you can see in this result, only the coefficients are negative for the |10> and |11>.
This means that the left side (the first qubit) is hurting the two ones with 1.

Conversely, if we apply a Z-gate, we know that the right side (the second qubit) will hurt two 1's.
Contrast this with a control Z-gate, which is the difference between hurting only one and hurting two.


Now the new gate we will learn about today is the control X-gate.
Let's first examine how it works.
It doesn't make sense to have the control qubit at |0>, so we'll leave it at |1>.
To do so, we can either implement it in initialization or multiply it by X in advance.

In [ ]:
qc2 = cirq.Circuit()
q = cirq.LineQubit.range(2)

qc2.append(cirq.X(q[0]))
qc2.append(cirq.CNOT(q[0], q[1]))

If you want to see what the circuit looks like, run print or SVGCircuit.

In [ ]:
#print(qc2)
from cirq.contrib.svg import SVGCircuit
SVGCircuit(qc2)

Let's try a quick simulation and see what results we get.

Since you will be running the simulation many times, it is a good idea to consolidate the parts of the simulation that run the simulation with your own functions.

In [ ]:
import numpy as np
def sim_state(qc,disp=True):
  res = sim.simulate(qc)
  if disp == True:
    print(cirq.dirac_notation(np.array(res.final_state_vector)))
  return res

This function has an optional variable called disp.
If nothing is entered, the function outputs a state vector in ket display.

In [ ]:
state = sim_state(qc2)

Type disp = False to turn off the display.

In [ ]:
state = sim_state(qc2, disp = False)

Now, if you look at the result, you can see how the |10> has turned into a |11>.
In other words, if the control qubit is a|1>, we do an inversion.
When you input the|11>, it becomes|10>.

In other words.  
|00> -> |00>,  
|01> -> |01>,  
|10> -> |11>,  
|11> -> |10>,  

and so on, the bottom two states are swapped.

This can be used to change the action of the Z gate that scratches the two.
To try it out, we will sandwich the Z-gate with a control X-gate.

To compare how it will act at once, we will implement the Hadamard circuit first.

In [ ]:
qc3 = cirq.Circuit()
q = cirq.LineQubit.range(2)

qc3.append(cirq.H.on_each(q))

qc3.append(cirq.CNOT(q[0], q[1]))
qc3.append(cirq.Z(q[1]))
qc3.append(cirq.CNOT(q[0], q[1]))

Let's take a look at the circuit.

In [ ]:
SVGCircuit(qc3)

Now how will this work? Let's look at the results as we make our predictions.


In [ ]:
state = sim_state(qc3)

Only the|01> or|10> are scratched, while the rest of the numbers|00> and|11> are left untouched.
This can be said to determine whether the number of 1's is odd or even.
The number relationship between two qubits determines the result, so we say they are interrelated or interacting with each other.

This is important describing the Ising model, which also appears in the field of quantum computing.
The Z-gate was to rotate 180 degrees around the Z-axis.
Let's instead use a rotating gate that does not rotate up to 180 degrees around the Z-axis, but allows for fine tuning.

In [ ]:
theta = 0.5

qc4 = cirq.Circuit()
q = cirq.LineQubit.range(2)

qc4.append(cirq.H.on_each(q))

qc4.append(cirq.CNOT(q[0], q[1]))
qc4.append(cirq.rz(theta).on(q[1]))
qc4.append(cirq.CNOT(q[0], q[1]))

Let's take a look at the circuit.

In [ ]:
SVGCircuit(qc4)

There are others rx,ry,rz, which can be used with rz (angle, specifying the qubit).
Let's take a quick look at the results.

In [ ]:
state = sim_state(qc4)

The same coefficients are applied to|00> and|11>, and
The same coefficients are applied differently (though complex conjugate) to|01> and|10>.
The fact remains that the action changes depending on whether the number of 1's is even or odd.
As the angle is increased as the degree of interaction, the two states are separated.
However, since the angle is of course a $2\pi$ period, they will repeatedly move closer and farther apart.

As I mentioned earlier, let's assume that the spin of a qubit is upward for the |0> state and downward for the |1> state, and let's assume that the spin of the qubit is downward for the |0> state and downward for the |1> state.
This can be represented by the Ising model, which is known as the model of a magnetic body (magnet).
In this way, a quantum computer can simulate what is happening inside a material and investigate its behavior.

When simulating what is happening in such matter, it becomes necessary to compare the results with the results of actual experiments.
Quantum mechanics determines what values such physical quantities will have, and quantum mechanics determines what will happen probabilistically.
Reflecting this, what predicts the result of a measurement is an expected value.
We will now show you how to calculate the expected value from the quantum state vector obtained as a result of the simulation.

In [ ]:
obs = [cirq.Z(q[0]), cirq.Z(q[1])]
y = sum(sim.simulate_expectation_values(qc, observables=obs))
print(y)

The expected value is the predicted average value obtained by performing this experiment many times.
Starting from the superposition state, the spin orientation changes according to the dynamics of the Ising model for a little bit, but it was only for a short time, so the change is not very significant.

### Quantum Simulation

The Ising model is represented by a controlled X-gate and a Z-rotation gate.
It simulates the interaction between qubits or between spins.
In addition, an X-rotation gate, sometimes called a transverse magnetic field, can be applied to each qubit to mimic what is happening in a quantum annealing machine.

The angle of rotation corresponds to its simulation time.
However, if the rotation is done all at once, the accuracy of the simulation will suffer.
Therefore, it is necessary to apply it little by little.
This method of simulation by rotating the machine little by little is called Suzuki Trotter decomposition.

Let us now prepare a quantum circuit that simulates what is done in the transverse magnetic field Ising model for a short period of time.

In [ ]:
class Ising_dynamics(cirq.Gate):

  def __init__(self, n, theta_z, theta_x):
    self.n = n
    #theta_xはnumpy array
    #theta_zもnumpy array
    self.theta_z = theta_z
    self.theta_x = theta_x

  def _num_qubits_(self):
    return self.n

  def _decompose_(self, qubits):
    q = qubits
 
    for k in range(self.n):
      yield cirq.rx(self.theta_x[k]).on(q[k])
    
    if self.n > 1:
      for k in range(self.n-1):
        yield cirq.CNOT(q[k],q[k+1])
        yield cirq.rz(self.theta_z[k+1]).on(q[k+1])
        yield cirq.CNOT(q[k],q[k+1])
      yield cirq.CNOT(q[self.n-1],q[0])
      yield cirq.rz(self.theta_z[0]).on(q[0])
      yield cirq.CNOT(q[self.n-1],q[0])

  def _circuit_diagram_info_(self, args):
    return ["UIsing"] * self.num_qubits()

Next, we want to find out how many qubits or spins are aligned as a result of the quantum simulation.
We will prepare a function to calculate the expectation value for this purpose.
For future reference, we have made it possible to give various rotation angles for each qubit.
Let's actually use this to run a quantum simulation of the transverse magnetic field Ising model.
From the results obtained, we should be able to see the results for the measured expectation value of the spin in the Z direction.

In [ ]:
def mag_exp(qc,n):
  obs = [cirq.Z(q[i]) for i in range(n)]
  y = np.mean(sim.simulate_expectation_values(qc, observables=obs))
  return y

Let's put together a quantum circuit that uses these to perform a quantum simulation.

In [ ]:
n = 5
#Number of steps
Tall = 50
#List to store intermediate progress
m_series = []

qc5 = cirq.Circuit()
q = cirq.LineQubit.range(n)

#Start with the super position
#qc5.append(cirq.H.on_each(q))

#Simulate in a short time period
dt = 0.1
#Transverse magnetic field magnitude
theta_x = 5.0*np.ones(n)*dt
#Magnitude of interaction
theta_z = 3.0*np.ones(n)*dt

UIsing = Ising_dynamics(n, theta_z, theta_x)
for k in range(Tall):
  qc5.append(UIsing.on(*q))
  m = mag_exp(qc5,n)
  m_series.append(m)


Now, how does the magnetization (spin alignment: the average value of the spins) change as a result of the quantum simulation?

In [ ]:
import matplotlib.pyplot as plt
plt.plot(m_series)
plt.show()

You can see that the behavior is quite complex.
It seems to start out all aligned and then gradually collapse, oscillating and repeatedly aligning and collapsing.

#### Using sympy
By the way, just to introduce a little more, sympy is a convenient way to perform calculations with the display in the textbook.

In [ ]:
from sympy.physics.quantum.qubit import Qubit
q = Qubit("01")
print(q)

Such a ket display can be made into a vector display.

In [ ]:
from sympy.physics.quantum.represent import represent
represent(q)

np.array(represent(q)), which can be used as a numpy array (matrix).

Conversely, it displays matrices (and vectors) represented as arrays in numpy as they are in ket vectors.

In [ ]:
from sympy.physics.quantum.qubit import matrix_to_qubit
matrix_to_qubit(represent(q))

### Go to Quantum Machine Learning

Now for today's main topic.

In (supervised) machine learning, we have an output y for an input x, and the goal is to mimic a function f(x) that connects the two.
For example, given an image of a cat as x, the idea is to create a function that gives y whether it is a cat or not.
For example, if y=+1 to indicate that it is a cat and y=-1 to indicate that it is not a cat, we can imagine it as a nice function.
But we don't know what kind of function it is.
So we try to create a well-consistent function by combining functions of our choice and adjusting the way they are combined.
Along the way, we will include an actual cat image to check if we have correctly identified the cat $y=\pm 1$.
If it is not done well, we change the elements involved in the combination.

Let's use the quantum circuits that we are learning with you in this project for the part of preparing the function of your choice.

Let's start by creating a simple circuit that allows a quantum circuit to input an image of a cat, such that the result changes depending on the input x.
Since what a quantum circuit can do is basically "rotation," we need to change the numerical value of the input into an angle of rotation; let's use trigonometric functions such as cos and sin. These trigonometric functions have angles from 0 to 360 degrees ($0$ to $2\pi$ in the arc degree method) and their values vary from -1 to 1. Using this in reverse, they change the value of -1 to +1 from 0 to 360 degrees ($0$ to $2\pi$ in the arc degree method).
So it turns the input value into a rotation angle.
Let's prepare a circuit to just turn it that way.

In [ ]:
class U_in(cirq.Gate):
  def __init__(self, x, n):
    self.x = x
    self.n = n

  def _num_qubits_(self):
    return self.n

  def _decompose_(self, qubits):
    q = qubits
    angle = np.arcsin(self.x)
    yield cirq.rx(angle).on_each(q)

  def _circuit_diagram_info_(self, args):
    return ["U_in"] * self.num_qubits()

Now let's look at how the probability of a quantum state arising as the input changes from -1 to +1 changes.

First, let's make a program that gives an input x, generates a quantum circuit for it, even measures it, and calculates the probability of how many states of 0 will occur.

In [ ]:
def QCLinput(x,n):
  qc = cirq.Circuit()
  q = cirq.LineQubit.range(n)

  u_in = U_in(x,n)
  qc.append(u_in.on(*q))
  
  return qc

When this is done, a quantum circuit is created according to the input x.
If you read the output from that quantum circuit, you will get some value output from the quantum state that is transformed according to the input.
From the resulting quantum state, let's examine the alignment of the quantum bits and the alignment of the spins in the z-direction that we used in the quantum simulation earlier.

In [ ]:
n = 3
x = 0.1
q = cirq.LineQubit.range(n)
qc = QCLinput(x,n)
y = mag_exp(qc,n)
print(y)

Now we have created a quantum circuit that produces an output y when x is input.
Let's run this one after another and see how y changes when x is changed.

In [ ]:
import numpy as np

x_series = np.linspace(-1,1,100)
y_series = []

for x in x_series:
  qc = QCLinput(x,n)
  y = mag_exp(qc,n)
  y_series.append(y)

Let's plot and view these results.
To do this, we will use matplotlib from the python library.

In [ ]:
import matplotlib.pyplot as plt
plt.plot(x_series,y_series)
plt.show()

We were able to create a beautiful semicircle. This is the result of using trigonometric functions.
For the input x, the output y is a semicircle, which means that we have created a function below.
\begin{equation}
y = \sqrt{1-x^2}
\end{equation}

This means that we have created a function that is a semi-circle.

We need to be prepared to create other clever functions to account for the input x and output y in various data.

In a quantum circuit, you can essentially rotate each qubit.
Also, by linking the qubits together, the coefficients can be manipulated, as in Grover's algorithm.
It seems possible to assign multiple qubits to a single input x to produce complex functions.



First, given multiple qubits, prepare a quantum circuit that rotates each of them appropriately.

In [ ]:
class U_rot(cirq.Gate):
  def __init__(self, n, params):
    self.n = n
    self.params = params

  def _num_qubits_(self):
    return self.n

  def _decompose_(self, qubits):
    q = qubits

    for k in range(self.n):
      yield cirq.rx(self.params[k]).on(q[k])
      yield cirq.ry(self.params[self.n+k]).on(q[k])
      yield cirq.rz(self.params[2*self.n+k]).on(q[k])

  def _circuit_diagram_info_(self, args):
    return ["U_rot"] * self.num_qubits()

We have added the ability to manipulate the rotation angle in the form of params here.

Now we can create various functions by adjusting the angle of rotation.
We can now change the shape of the function by adjusting the angle.

Just adding this will of course change the quantum state.
Let's see what kind of change it can bring about.
To get the output y, we need to compile the results from multiple qubits.
To do this, let's use the method of calculating the expected value of magnetization used in the quantum simulation.

The resulting quantum circuit stochastically outputs -1 and 1 for each qubit.
They are then integrated into a single result.
In this case, we can also consider which qubits are important, and from the similarity of neural networks, we can prepare parameters as weights.
Now you are ready. Let's see how the relationship between input x and output y changes by applying the appropriate rotation to each qubit.

In [ ]:
y_series = []
params = np.random.rand(3*n)*2*np.pi

u_rot = U_rot(n,params)
for x in x_series:
  qc = QCLinput(x,n)
  q = cirq.LineQubit.range(n)
  qc.append(u_rot.on(*q))
  y = mag_exp(qc,n)
  y_series.append(y)

Here we have prepared params as a random parameter.

In [ ]:
plt.plot(x_series,y_series)
plt.show()

Next, let's add an operation to change the coefficients between qubits with a control Z-gate.

In [ ]:
class U_ent(cirq.Gate):
  def __init__(self, n):
    self.n = n

  def _num_qubits_(self):
    return self.n

  def _decompose_(self, qubits):
    q = qubits
    if self.n > 1:
      for k in range(self.n-1):
        yield cirq.CZ(q[k],q[k+1])
      yield cirq.CZ(q[self.n-1],q[0])

  def _circuit_diagram_info_(self, args):
    return ["U_ent"] * self.num_qubits()

How would the combination of these changes affect the results?

In [ ]:
y_series = []

u_rot = U_rot(n,params)
u_ent = U_ent(n)

for x in x_series:
  qc = QCLinput(x,n)
  q = cirq.LineQubit.range(n)
  qc.append(u_ent.on(*q))
  qc.append(u_rot.on(*q))
  y = mag_exp(qc,n)
  y_series.append(y)

Now, what are the results?

In [ ]:
plt.plot(x_series,y_series)
plt.show()

It was as if the functional form was different from the previous one.
The control Z-gate had the effect of scratching only the |11>.
I wonder if that is the effect.

Just repeating this might make the function somewhat more complex.

In [ ]:
y_series = []

depth = 3
params = np.random.rand(3*depth*n)

u_ent = U_ent(n)

for x in x_series:
  qc = QCLinput(x,n)
  q = cirq.LineQubit.range(n)
  for d in range(depth):
    qc.append(u_ent.on(*q))
    u_rot = U_rot(n,params[d*3*n:(d+1)*3*n])
    qc.append(u_rot.on(*q))
  y = mag_exp(qc,n)
  y_series.append(y)

In [ ]:
plt.plot(x_series,y_series)
plt.show()

The shape has changed again.

However, this was the result of randomly assigning parameters, and without specifying what they should match.
In machine learning, the goal is to estimate the form of the function given the data in advance and then follow it.
So let's prepare the data.

In [ ]:
ntrain = 10
func = lambda x: 0.5*x**3
xtrain = 2*np.random.rand(ntrain)-1
ytrain = func(xtrain)

In [ ]:
plt.scatter(xtrain,ytrain)

The goal is to infer the form of the original function (func in this case) from such fragmentary information.

The goal is to move random parameters to match the data we receive.
Therefore, it is necessary to clarify how different the data and the results that the quantum circuit has played out are.

In [ ]:
def cost_func(params):
    u_ent = U_ent(n)
    cost_total = 0
    for k in range(ntrain):
      qc = QCLinput(xtrain[k],n)
      q = cirq.LineQubit.range(n)
      for d in range(depth):
        qc.append(u_ent.on(*q))
        u_rot = U_rot(n,params[d*3*n:(d+1)*3*n])
        qc.append(u_rot.on(*q))
      y = mag_exp(qc,n)

      #Calculate the discrepancy between the data and the output result of the quantum circuit
      cost = 0.5*(ytrain[k] - y)**2
      cost_total += cost

    #Calculate the average error by dividing by the number of training data
    cost_total = cost_total/ntrain

    return cost_total

This cost function is "optimized" to be as small as possible by varying the parameters.
This is to match the data as closely as possible.
Let's try it.

In [ ]:
#warningが多発する場合に抑制するコマンド
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from scipy.optimize import minimize
result = minimize(cost_func, params, method="COBYLA", options={"maxiter": 100})

If you want to see how close you are getting, it is result.fun.

In [ ]:
result.fun

To retrieve the result, type result.x.

In [ ]:
result.x

This is the set of parameters resulting from the optimization attempts for the specified number of times (maxiter).

Let's plot a graph using these results.

In [ ]:
y_series = []
params = result.x
u_ent = U_ent(n)
for x in x_series:
  qc = QCLinput(x,n)
  q = cirq.LineQubit.range(n)
  for d in range(depth):
    qc.append(u_ent.on(*q))
    u_rot = U_rot(n,params[d*3*n:(d+1)*3*n])
    qc.append(u_rot.on(*q))
  y = mag_exp(qc,n)
  y_series.append(y)

Let's also overlay the data and write the original function on top of it, since we're at it.

In [ ]:
y_correct = func(x_series)

In [ ]:
plt.scatter(xtrain,ytrain)
plt.plot(x_series,y_correct)
plt.plot(x_series,y_series)
plt.show()

What do you think?
Did you get it right?

Let us keep the quantum circuit in the deep direction thus created as a function.

In [ ]:
def forward(x,n,depth,params):
    qc = QCLinput(x,n)
    q = cirq.LineQubit.range(n)
    u_ent = U_ent(n)
    for d in range(depth):
      qc.append(u_ent.on(*q))
      u_rot = U_rot(n,params[d*3*n:(d+1)*3*n])
      qc.append(u_rot.on(*q))
    y = mag_exp(qc,n)
    return y

And by the way, all you need to do to fit the data in machine learning is to figure out
It is all about how to make non-trivial movements and how to integrate those non-trivial movements.
So far, we have used only control z-gates and rotations, which is simple.
To make it more complex here, let's use the quantum simulation of the Ising model we just used as a test.

In [ ]:
#Transverse magnetic field magnitude
theta_x = np.random.randn(n)
#Magnitude of interaction
theta_z = np.random.randn(n)

def cost_func2(params):
    UIsing = Ising_dynamics(n, theta_z, theta_x)

    cost_total = 0
    for k in range(ntrain):
      qc = QCLinput(xtrain[k],n)
      q = cirq.LineQubit.range(n)
      for d in range(depth):
        qc.append(UIsing.on(*q))
        u_rot = U_rot(n,params[d*3*n:(d+1)*3*n])
        qc.append(u_rot.on(*q))
      y = mag_exp(qc,n)

      #Calculate the average error by dividing by the number of training data
      cost = 0.5*(ytrain[k] - y)**2
      cost_total += cost

    #Calculate the average error by dividing by the number of training data
    cost_total = cost_total/ntrain

    return cost_total

In [ ]:
params = np.random.rand(3*depth*n)*2*np.pi
result = minimize(cost_func2, params, method="COBYLA", options={"maxiter": 100})

To see how close you could get to the same result.fun

In [ ]:
result.fun

To see the resulting parameters, it's result.x.

In [ ]:
result.x

If you want to iterate the optimization a bit more, continue with result.x as the initial solution.

In [ ]:
#result = minimize(cost_func2, result.x, method="COBYLA", options={"maxiter": 100})

Compare this result to the data.

In [ ]:
y_series = []
params = result.x

UIsing = Ising_dynamics(n, theta_z, theta_x)

for x in x_series:
  qc = QCLinput(x,n)
  q = cirq.LineQubit.range(n)
  for d in range(depth):
    qc.append(UIsing.on(*q))
    u_rot = U_rot(n,params[d*3*n:(d+1)*3*n])
    qc.append(u_rot.on(*q))
  y = mag_exp(qc,n)
  y_series.append(y)

In [ ]:
plt.scatter(xtrain,ytrain)
plt.plot(x_series,y_correct)
plt.plot(x_series,y_series)
plt.show()

However, the method used here (COBYLA, Nelder-Mead, Powell, etc. can also be used for optimization) is not appropriate for optimization without using gradients.
It would be more reliable to use gradient as in deep learning and proceed with the computation efficiently.

The usual way in machine learning is to calculate the gradient by slightly shifting the parameters and differentiating them.
In the case of quantum circuits, by using the parameters to be used and the characteristics of the quantum circuit with those parameters, such as the angle of rotation, it is possible to calculate the same result as the calculation of the derivative by subtracting the $\pi/2$ shifted and the $-\pi/2$ shifted.

However, for the differentiation, it is necessary to perform the calculations many times, which is very computationally intensive, because it requires the results of calculations by the quantum circuit with the current parameters and the quantum circuit with shifted parameters.

In [ ]:
def calc_grad(n,depth,xtrain,ytrain,params):    
  grad = np.zeros_like(params)
  cost_data = np.zeros(ntrain)
  shifted = params.copy()
    
  for k in range(ntrain):
    x = xtrain[k]
    y = forward(x,n,depth,params)
    cost_data[k] = - (ytrain[k] - y)
    
    for i in range(len(params)):
      shifted[i] += np.pi/2
      y1 = forward(x,n,depth,shifted)    
      shifted[i] -= np.pi
      y2 = forward(x,n,depth,shifted)    
      gradient = 0.5 * (y1-y2)
      grad[i] += cost_data[k]*gradient/ntrain
      shifted[i] += np.pi/2

  return grad, cost_data

In [ ]:
params = np.random.rand(3*depth*n)*2*np.pi

eta = 1.0
Tall = 20
cost_series = []

In [ ]:
for t in range(Tall):
  grad, cost_data = calc_grad(n,depth,xtrain,ytrain,params)
  
  cost = np.sum(cost_data**2)/len(cost_data)
  cost_series.append(cost)
  
  params = params - eta*grad

In [ ]:
plt.plot(cost_series)
plt.show()

In [ ]:
y_series = []

for x in x_series:
  y = forward(x,n,depth,params)
  y_series.append(y)

In [ ]:
plt.scatter(xtrain,ytrain)
plt.plot(x_series,y_correct)
plt.plot(x_series,y_series)
plt.show()